##Pruebas recuperación numericas [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mariaprojas/AI-Project/blob/master/05-pruebas%20recuperación%20numericas.ipynb)



In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from progressbar import progressbar as pbar
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
!pip install fancyimpute
from fancyimpute import IterativeImputer
from sklearn.experimental import enable_hist_gradient_boosting  # necesario para usar HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 14.4 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29898 sha256=b0cbefa31a1a3b500386b2fc970501b84392bd408349bfc9560d076069655477
  Stored in directory: /root/.cache/pip/wheels/f9/fc/6a/b0406b906bce293abe23c3b6da5a72637d2d04146ef1125a0b
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11352 sha256=8d07ee52409c94821cd35695223e7b97f72037541847650d8d4c0cf4168fb62b
  Stored in directory: /root/.cache/pip/wheels/88/c4/be/e232c750d9bc360abf9a5e2cafe0d3e08e3605d2801bb11684
Successfully built fancyimpute knnimpute


/usr/local/lib/python3.9/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [ ]:
!pip install gdown

import gdown

# Define ID 
archivo_id = "1-4cFfd9pgcVVut2KRvyIRNITLfZD2hyZ"

# Asigna nombre para el archivo en Colab
d1 = "dataset2.csv"

# Construye el enlace de descarga del archivo
enlace_descarga = "https://drive.google.com/uc?id=" + archivo_id

# Descarga el archivo desde el enlace de descarga y lo guarda en Colab
gdown.download(enlace_descarga, d1, quiet=False)

# Lee el archivo y lo almacena en un data frame
d = pd.read_csv("dataset2.csv") 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading...
From: https://drive.google.com/uc?id=1-4cFfd9pgcVVut2KRvyIRNITLfZD2hyZ
To: /content/dataset2.csv
100%|██████████| 383M/383M [00:04<00:00, 92.0MB/s]


In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from scipy.stats import norm

# leer el archivo csv
df = d

# método 1: llenar los NaN en columnas numéricas con la media
num_cols = df.select_dtypes(include=np.number).columns
df_mean = df.copy()
for col in num_cols:
    col_mean = df[col].mean()
    df_mean[col] = df[col].fillna(col_mean)

# método 2: llenar los NaN restantes con la interpolación exponencial
df_exp = df.copy()
exp_cols = [col for col in df.select_dtypes(include=np.number).columns if df[col].isna().sum() > 0]
for col in exp_cols:
    x = np.arange(len(df_exp[col]))
    y = df_exp[col].values
    f = interp1d(x, y, kind='linear', bounds_error=False, fill_value='extrapolate')
    df_exp[col] = pd.Series(f(x))

# método 3: llenar los NaN restantes con la distribución Gaussiana
df_gauss = df.copy()
gauss_cols = [col for col in df.select_dtypes(include=np.number).columns if df[col].isna().sum() > 0 and df[col].nunique() > 1]
for col in gauss_cols:
    col_mean = df_gauss[col].mean()
    col_std = df_gauss[col].std()
    df_gauss[col] = pd.Series(norm.rvs(loc=col_mean, scale=col_std, size=len(df_gauss)))

# método 4: llenar los NaN en columnas numéricas con la mediana
num_cols = df.select_dtypes(include=np.number).columns
df_median = df.copy()
for col in num_cols:
    col_median = df[col].median()
    df_median[col] = df[col].fillna(col_median)


# método 5: llenar los NaN restantes con una muestra aleatoria de la columna
df_random = df.copy()
for col in num_cols:
    missing = df_random[col].isna()
    num_missing = missing.sum()
    if num_missing > 0:
        random_sample = np.random.choice(df_random[col][~missing], size=num_missing, replace=True)
        df_random.loc[missing, col] = random_sample

# método 6: llenar los NaN con el valor anterior o siguiente
df_ffill_bfill = df.copy()
for col in num_cols:
    df_ffill_bfill[col].fillna(method='ffill', inplace=True)
    df_ffill_bfill[col].fillna(method='bfill', inplace=True)

In [ ]:
d

,Unnamed: 0,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,AVProductStatesIdentifier,HasTpm,CountryIdentifier,...,Census_ActivationChannel,Census_IsFlightsDisabled,Census_FlightRing,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,HasDetections
0,0,f483b33ca4c1148aa8f7325fa553d274,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1283.0,0.0,46669.0,1.0,166.0,...,OEM:DM,0.0,Retail,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,f483b62fac66c56ad4359fbd56b2d7f2,win8defender,1.1.15000.2,4.9.10586.0,1.271.1198.0,0.0,53447.0,1.0,162.0,...,Retail,0.0,Retail,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,f483b882ef1a55e2154e73748f05eb5b,win8defender,1.1.14500.5,4.11.15063.447,1.261.1325.0,NaN,51954.0,NaN,44.0,...,OEM:DM,0.0,Retail,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,f483b9cfa302a83424696eaa044dd9df,win8defender,1.1.15300.6,4.12.16299.15,1.277.18.0,0.0,43927.0,1.0,155.0,...,OEM:DM,0.0,Retail,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4,4,f483b9dfefe07cacf089d900a3379bf3,win8defender,1.1.15200.1,4.18.1807.18075,1.275.323.0,0.0,51954.0,1.0,41.0,...,OEM:DM,0.0,Retail,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,799995,1cb96260a4e91a314545ee86bf82b55d,win8defender,1.1.15100.1,4.12.16299.15,1.273.1188.0,0.0,53447.0,1.0,214.0,...,OEM:DM,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,1.0
799996,799996,1cb96272df74bb640b139621672ab18f,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1140.0,0.0,53447.0,1.0,153.0,...,OEM:DM,0.0,Retail,1.0,0.0,0.0,0.0,0.0,0.0,1.0
799997,799997,NaN,win8defender,1.1.14901.4,4.16.17656.18052,1.269.224.0,0.0,62773.0,1.0,93.0,...,Retail,0.0,Retail,NaN,NaN,1.0,0.0,0.0,0.0,0.0
799998,799998,NaN,win8defender,1.1.15200.1,4.18.1807.18075,1.275.727.0,0.0,53447.0,1.0,66.0,...,OEM:NONSLP,0.0,Retail,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [ ]:
df_gauss

,Unnamed: 0,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,AVProductStatesIdentifier,HasTpm,CountryIdentifier,...,Census_ActivationChannel,Census_IsFlightsDisabled,Census_FlightRing,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,HasDetections
0,0,f483b33ca4c1148aa8f7325fa553d274,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1283.0,-0.002967,47391.547209,0.939426,0.444910,...,OEM:DM,0.004440,Retail,0.460466,0.028388,0.275553,0.124317,0.008382,0.249256,0.634422
1,1,f483b62fac66c56ad4359fbd56b2d7f2,win8defender,1.1.15000.2,4.9.10586.0,1.271.1198.0,0.004739,30432.198484,0.906372,81.997774,...,Retail,-0.002834,Retail,0.620365,-0.098858,0.101702,-0.345142,-0.183743,0.572033,1.072752
2,2,f483b882ef1a55e2154e73748f05eb5b,win8defender,1.1.14500.5,4.11.15063.447,1.261.1325.0,-0.004103,55781.088854,0.957893,56.518864,...,OEM:DM,0.000564,Retail,0.543893,0.033283,0.238463,0.164971,-0.007199,-0.033543,0.516284
3,3,f483b9cfa302a83424696eaa044dd9df,win8defender,1.1.15300.6,4.12.16299.15,1.277.18.0,-0.002546,47273.955317,0.853297,163.808332,...,OEM:DM,0.000126,Retail,1.173818,-0.060668,0.033471,0.133728,0.198918,0.272864,1.207683
4,4,f483b9dfefe07cacf089d900a3379bf3,win8defender,1.1.15200.1,4.18.1807.18075,1.275.323.0,0.003742,61754.824353,0.842787,82.845761,...,OEM:DM,0.001003,Retail,-0.042047,0.067740,0.272112,-0.140183,-0.087162,0.162543,-0.192860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,799995,1cb96260a4e91a314545ee86bf82b55d,win8defender,1.1.15100.1,4.12.16299.15,1.273.1188.0,0.002383,41836.040873,0.978779,104.953984,...,OEM:DM,-0.001293,NaN,0.897280,-0.039933,-0.110075,0.067804,0.010085,-0.624738,0.180219
799996,799996,1cb96272df74bb640b139621672ab18f,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1140.0,0.000082,47540.968390,1.140596,95.333772,...,OEM:DM,-0.001678,Retail,0.748242,-0.053617,0.190954,-0.039541,0.060893,-0.772579,0.231714
799997,799997,NaN,win8defender,1.1.14901.4,4.16.17656.18052,1.269.224.0,0.001485,53557.190039,0.957640,139.648415,...,Retail,-0.000824,Retail,1.148574,0.007704,-0.111751,-0.389031,0.533389,0.072360,0.048132
799998,799998,NaN,win8defender,1.1.15200.1,4.18.1807.18075,1.275.727.0,-0.000233,65422.987001,0.874997,70.217595,...,OEM:NONSLP,-0.001301,Retail,0.913356,0.182411,0.703327,0.327961,0.184862,0.422333,0.125937


In [ ]:
df_exp 

,Unnamed: 0,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,AVProductStatesIdentifier,HasTpm,CountryIdentifier,...,Census_ActivationChannel,Census_IsFlightsDisabled,Census_FlightRing,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,HasDetections
0,0,f483b33ca4c1148aa8f7325fa553d274,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1283.0,0.0,46669.0,1.0,166.0,...,OEM:DM,0.0,Retail,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,f483b62fac66c56ad4359fbd56b2d7f2,win8defender,1.1.15000.2,4.9.10586.0,1.271.1198.0,0.0,53447.0,1.0,162.0,...,Retail,0.0,Retail,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,f483b882ef1a55e2154e73748f05eb5b,win8defender,1.1.14500.5,4.11.15063.447,1.261.1325.0,NaN,51954.0,NaN,44.0,...,OEM:DM,0.0,Retail,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,f483b9cfa302a83424696eaa044dd9df,win8defender,1.1.15300.6,4.12.16299.15,1.277.18.0,NaN,43927.0,NaN,155.0,...,OEM:DM,0.0,Retail,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4,4,f483b9dfefe07cacf089d900a3379bf3,win8defender,1.1.15200.1,4.18.1807.18075,1.275.323.0,0.0,51954.0,1.0,41.0,...,OEM:DM,0.0,Retail,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,799995,1cb96260a4e91a314545ee86bf82b55d,win8defender,1.1.15100.1,4.12.16299.15,1.273.1188.0,0.0,53447.0,NaN,214.0,...,OEM:DM,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaN,1.0
799996,799996,1cb96272df74bb640b139621672ab18f,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1140.0,0.0,53447.0,1.0,153.0,...,OEM:DM,0.0,Retail,1.0,0.0,0.0,0.0,0.0,NaN,1.0
799997,799997,NaN,win8defender,1.1.14901.4,4.16.17656.18052,1.269.224.0,0.0,62773.0,1.0,93.0,...,Retail,0.0,Retail,NaN,NaN,1.0,0.0,0.0,0.0,0.0
799998,799998,NaN,win8defender,1.1.15200.1,4.18.1807.18075,1.275.727.0,0.0,53447.0,1.0,66.0,...,OEM:NONSLP,0.0,Retail,NaN,NaN,0.0,0.0,0.0,1.0,1.0


In [ ]:
df_mean

,Unnamed: 0,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,AVProductStatesIdentifier,HasTpm,CountryIdentifier,...,Census_ActivationChannel,Census_IsFlightsDisabled,Census_FlightRing,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,HasDetections
0,0,f483b33ca4c1148aa8f7325fa553d274,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1283.0,0.000000,46669.0,1.000000,166.0,...,OEM:DM,0.0,Retail,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
1,1,f483b62fac66c56ad4359fbd56b2d7f2,win8defender,1.1.15000.2,4.9.10586.0,1.271.1198.0,0.000000,53447.0,1.000000,162.0,...,Retail,0.0,Retail,1.000000,0.000000,0.0,0.0,0.0,0.000000,1.0
2,2,f483b882ef1a55e2154e73748f05eb5b,win8defender,1.1.14500.5,4.11.15063.447,1.261.1325.0,0.000006,51954.0,0.987862,44.0,...,OEM:DM,0.0,Retail,1.000000,0.000000,1.0,0.0,0.0,0.000000,0.0
3,3,f483b9cfa302a83424696eaa044dd9df,win8defender,1.1.15300.6,4.12.16299.15,1.277.18.0,0.000000,43927.0,1.000000,155.0,...,OEM:DM,0.0,Retail,1.000000,0.000000,1.0,0.0,0.0,0.000000,1.0
4,4,f483b9dfefe07cacf089d900a3379bf3,win8defender,1.1.15200.1,4.18.1807.18075,1.275.323.0,0.000000,51954.0,1.000000,41.0,...,OEM:DM,0.0,Retail,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,799995,1cb96260a4e91a314545ee86bf82b55d,win8defender,1.1.15100.1,4.12.16299.15,1.273.1188.0,0.000000,53447.0,1.000000,214.0,...,OEM:DM,0.0,NaN,1.000000,0.000000,0.0,0.0,0.0,0.283807,1.0
799996,799996,1cb96272df74bb640b139621672ab18f,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1140.0,0.000000,53447.0,1.000000,153.0,...,OEM:DM,0.0,Retail,1.000000,0.000000,0.0,0.0,0.0,0.000000,1.0
799997,799997,NaN,win8defender,1.1.14901.4,4.16.17656.18052,1.269.224.0,0.000000,62773.0,1.000000,93.0,...,Retail,0.0,Retail,0.486142,0.006969,1.0,0.0,0.0,0.000000,0.0
799998,799998,NaN,win8defender,1.1.15200.1,4.18.1807.18075,1.275.727.0,0.000000,53447.0,1.000000,66.0,...,OEM:NONSLP,0.0,Retail,0.000000,0.000000,0.0,0.0,0.0,1.000000,1.0


In [ ]:
df_random

,Unnamed: 0,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,AVProductStatesIdentifier,HasTpm,CountryIdentifier,...,Census_ActivationChannel,Census_IsFlightsDisabled,Census_FlightRing,Census_IsSecureBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,HasDetections
0,0,f483b33ca4c1148aa8f7325fa553d274,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1283.0,0.0,46669.0,1.0,166.0,...,OEM:DM,0.0,Retail,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,f483b62fac66c56ad4359fbd56b2d7f2,win8defender,1.1.15000.2,4.9.10586.0,1.271.1198.0,0.0,53447.0,1.0,162.0,...,Retail,0.0,Retail,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,f483b882ef1a55e2154e73748f05eb5b,win8defender,1.1.14500.5,4.11.15063.447,1.261.1325.0,0.0,51954.0,1.0,44.0,...,OEM:DM,0.0,Retail,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,3,f483b9cfa302a83424696eaa044dd9df,win8defender,1.1.15300.6,4.12.16299.15,1.277.18.0,0.0,43927.0,1.0,155.0,...,OEM:DM,0.0,Retail,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4,4,f483b9dfefe07cacf089d900a3379bf3,win8defender,1.1.15200.1,4.18.1807.18075,1.275.323.0,0.0,51954.0,1.0,41.0,...,OEM:DM,0.0,Retail,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,799995,1cb96260a4e91a314545ee86bf82b55d,win8defender,1.1.15100.1,4.12.16299.15,1.273.1188.0,0.0,53447.0,1.0,214.0,...,OEM:DM,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,1.0
799996,799996,1cb96272df74bb640b139621672ab18f,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1140.0,0.0,53447.0,1.0,153.0,...,OEM:DM,0.0,Retail,1.0,0.0,0.0,0.0,0.0,0.0,1.0
799997,799997,NaN,win8defender,1.1.14901.4,4.16.17656.18052,1.269.224.0,0.0,62773.0,1.0,93.0,...,Retail,0.0,Retail,0.0,0.0,1.0,0.0,0.0,0.0,0.0
799998,799998,NaN,win8defender,1.1.15200.1,4.18.1807.18075,1.275.727.0,0.0,53447.0,1.0,66.0,...,OEM:NONSLP,0.0,Retail,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
rmse = np.sqrt(mean_squared_error(d.fillna(0), df_random.fillna(0)))
print("RMSE:", rmse)

ValueError: ignored

In [ ]:
df_real = d

# Generar un dataframe de predicciones (usando datos aleatorios en este ejemplo)
df_pred = df_mean

corr_matrix = pd.concat([df_real, df_pred], axis=1).corr()

# obtener coeficiente de correlación promedio entre todas las columnas
coef = corr_matrix.mean().mean()
coef

<ipython-input-34-6d03abe82f05>:6: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = pd.concat([df_real, df_pred], axis=1).corr()


0.042589430328922935

In [ ]:
coef


0.042864776755450605

In [ ]:
# calcular el MSE
mse = mean_squared_error(data, data_imputed)
print('MSE:', mse)

# calcular el R²
r2 = r2_score(data, data_imputed)
print('R²:', r2)